In [37]:
import boto3
import sagemaker
from botocore.exceptions import BotoCoreError, ClientError
from contextlib import closing
import os
import sys
import subprocess
from tempfile import gettempdir
import IPython
import ipywidgets as widgets

sess = sagemaker.Session()
bucket = sess.default_bucket() 

In [13]:
polly = boto3.client("polly")

In [61]:
select_voice = {
    "us-female":  {
        "name": "Joanna",
        "text":   "It was November. Although it was not yet late, the sky was dark when I turned into Laundress Passage. " +
                      "Father had finished for the day, switched off the shop lights and closed the shutters; but so I would " +
                      "not come home to darkness he had left on the light over the stairs to the flat."
    },
    "es-male":  {
        "name": "Andres",
        "text": "Cozzella entrevistó a los nietos de los posaderos que, a finales de los años 40, tras la Segunda Guerra Mundial, " +
                    "daban de comer a los soldados estadounidenses en el pintoresco barrio de Trastevere, al otro lado del río Tíber, en Roma."
    },
    "es-female": {
        "name": "Lucia",
        "text": "Al parecer, los soldados pedían un 'desayuno espagueti': huevos, tocineta y pasta. Y en aquella época, aunque el país estaba de 'rodillas', " +
                    "los italianos podían comprar raciones militares en el mercado negro, que incluían tocino de los estadounidenses y huevo en polvo de los británicos."
    },
    "us-female-2":  {
        "name": "Emma",
        "text":   " Soldiers were apparently asking for spaghetti breakfast: eggs, bacon and pasta. And at the time, although the country was on its knees, " + 
                      "Italians could buy military rations on the black market that included bacon from Americans and egg powder from the British."
    },
}

In [63]:
select = widgets.Dropdown(
        options=[('us-female', select_voice['us-female']), ('es-male', select_voice['es-male']), ('es-female' ,select_voice['es-female' ]), ('us-female-2' ,select_voice['us-female-2' ])],
        value=select_voice['us-female'],
        description='Number:',
)
select

Dropdown(description='Number:', options=(('us-female', {'name': 'Joanna', 'text': 'It was November. Although i…

In [64]:
try:
    # Request speech synthesis
    response = polly.synthesize_speech(Text=select.value['text'], OutputFormat="mp3",
                                        VoiceId=select.value['name'], Engine='neural')
except (BotoCoreError, ClientError) as error:
    # The service returned an error, exit gracefully
    print("ERROR:", error)

In [65]:
# Access the audio stream from the response
if "AudioStream" in response:
    # Note: Closing the stream is important because the service throttles on the
    # number of parallel connections. Here we are using contextlib.closing to
    # ensure the close method of the stream object will be called automatically
    # at the end of the with statement's scope.
        with closing(response["AudioStream"]) as stream:
            output =  "speech.mp3"

            try:
            # Open a file for writing the output as a binary stream
                with open(output, "wb") as file:
                    file.write(stream.read())
            except IOError as error:
                # Could not write to file, exit gracefully
                print(error)
                sys.exit(-1)

else:
    # The response didn't contain audio data, exit gracefully
    print("Could not stream audio")
    sys.exit(-1)

In [66]:
IPython.display.Audio(output)